<a href="https://colab.research.google.com/github/4d5645/AppliedDataAnalysisProblems_MIPT/blob/main/Spam_filter_for_SMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from __future__ import division, print_function

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Loading the dataset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
with open('/content/gdrive/MyDrive/Colab Notebooks/MIPT/smsspamcollection/SMSSpamCollection.txt', 'r') as f:
    data = f.read().splitlines()

Mounted at /content/gdrive



Let's prepare two lists for further work: a list of texts in the order they appear in the dataset and a list of class labels corresponding to them. We use 1 for spam and 0 for "not spam" as the class label.

In [18]:
labels = [data[i].split('\t')[0] for i in range(len(data))]
content = [data[i].split('\t')[1] for i in range(len(data))]
# print(labels)
# print(content)

In [4]:
binary_labels = np.zeros(len(labels))
for i in range(len(labels)):
  if labels[i] == 'ham':
    binary_labels[i] = 0
  else:
    binary_labels[i] = 1

In [5]:
vectorizer = CountVectorizer()
X= vectorizer.fit_transform(content)
X.shape

(5574, 8713)

In [6]:
pipeline = Pipeline([('count_vect', CountVectorizer()), ('log_reg', LogisticRegression())])

In [7]:
score = cross_val_score(pipeline, content, binary_labels, scoring='f1', cv=10).mean()
print(score)

0.9311052112181922


In [8]:
sms = [ "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
       "FreeMsg: Txt: claim your reward of 3 hours talk time",
       "Have you visited the last lecture on physics?",
       "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
       "Only 99$",
       ]

In [9]:
pipeline.fit(content, binary_labels)

Pipeline(steps=[('count_vect', CountVectorizer()),
                ('log_reg', LogisticRegression())])

In [10]:
test = pipeline.predict(sms)
test

array([1., 1., 0., 0., 0.])

In [12]:
ngrams = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngrams:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=ngram)), ('log_reg', LogisticRegression())])
    score = cross_val_score(pipeline, content, binary_labels, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.8168

Ngram_range:  (3, 3)
Cross-val-score f1: 0.7250

Ngram_range:  (1, 3)
Cross-val-score f1: 0.9223



In [16]:
scores = []
for ngram in ngrams:
    X_counts = CountVectorizer(ngram_range=ngram).fit_transform(content)
    mn_nb = MultinomialNB()
    score = cross_val_score(mn_nb, X_counts, binary_labels, scoring='f1', cv=10).mean()
    scores.append(score)
    print('Ngram_range: ', ngram)
    print('Cross-val-score f1: %.4f\n' % score)

Ngram_range:  (2, 2)
Cross-val-score f1: 0.6451

Ngram_range:  (3, 3)
Cross-val-score f1: 0.3786

Ngram_range:  (1, 3)
Cross-val-score f1: 0.8878



In [17]:
pipeline = Pipeline([('tf_idf', TfidfVectorizer()),
                     ('log_reg', LogisticRegression())])
score = cross_val_score(pipeline, content, binary_labels, scoring='f1', cv=10).mean()
print('Cross-val-score f1: %.4f\n' % score)

Cross-val-score f1: 0.8784

